### Imports

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.special import sph_harm
from matplotlib import animation
from matplotlib.colors import LightSource
from IPython import display
import matplotlib

from dataloader import loadhcp
from scipy.special import sph_harm
from preprocessing.data_augmentation import extend_dataset_with_origin_reflections
from preprocessing.data_cleaning import remove_b_0_measurements
from preprocessing.data_transformations import convert_coords_from_cartesian_to_spherical
from sphericalharmonics.spherical_fourier_transform import get_spherical_fourier_transform
from sphericalharmonics.spherical_fourier_transform import get_design_matrix
from sphericalharmonics.spherical_fourier_transform import get_inverse_spherical_fourier_transform

# Spherical harmonics

Spherical harmonics are special functions defined on the surface of a sphere. "Since the spherical harmonics form a complete set of orthogonal functions and thus an orthonormal basis, each function defined on the surface of a sphere can be written as a sum of these spherical harmonics." Spherical harmonics are the extension of Fourier series to higher dimensions. As sines and cosines in Fourier series forms an orthonormal basis for functions on the circle (S(1) group), spherical harmonics form an orthonormal basis for any spherical function (S(2) group)i.e. any function defined on the surface of a sphere can be written as a sum of spherical harmonics. Thus spherical harmonics provide a smooth representation of data distributed on a sphere. Similarly to the sines and cosines in Fourier series, the spherical harmonics can also be ordered by angular frequency (grouped in rows):

![SPHERICAL_HARMONICS_VISUALIZATION](./figures/spherical_harmonics_visualization.png)

"Further, spherical harmonics are basis functions for irreducible representations of SO(3), the group of rotations in three dimensions."

"Spherical harmonics emerge from solving Laplace's equation in a spherical domain."

Spherical harmonics of degree $l$ and order $m$, where $l\geq 0$ and $-l\leq m\leq l$ is defined as (Eq 1):

$$Y^{m}_{l}(\theta ,\phi )=\sqrt{\frac{(2l+1)}{4\pi }\frac{(l-m)!}{(l+m)!}}P^{m}_{l}(cos \, \theta)e^{im\phi }$$

Where $P^{m}_{l}$ is the associated Legendre polynomial defined as (Eq 2):

$$P^{m}_{l}(x)=(-1)^{m}(1-x^{2})^{\frac{m}{2}}\frac{d^{m}}{dx^{m}}P_{l}(x)$$

where (Eq 3):

$$P_{l}(x)=\sum_{k=0}^{\infty}\frac{(-v)_{k}(v+1)_{k}}{(k!)^{2}}\left ( \frac{1-x}{2} \right )^{k}$$

and $(\cdot)_{k}$ is the Pochhammer symbol defined as (Eq 4):

$$(x)_{k}=\frac{\Gamma(x+k)}{\Gamma(x)}$$

The order $m$ corresponds to the angular frequency of the basis function i.e. how many full oscilations do we encounter when going all the way around some equator (some constant latitude) on the sphere. And the degree $l$ corresponds to different orthogonal modes at given angular frequency $m$. For a given value of $l$, there are $2l + 1$ independent solutions of this form, one for each integer $m$.

The colatitude $\theta$, or polar angle, ranges from $0$ at the North Pole, to $\frac{\pi}{2}$ at the Equator, to $\pi$ at the South Pole, and the longitude $\phi$, or azimuth, may assume all values with $0 \leq \phi < 2\pi$.

In a spherical coordinate system, a colatitude is the complementary angle of a given latitude, i.e. the difference between a right angle and the latitude.[1] Here Southern latitudes are defined to be negative, and as a result the colatitude is a non-negative quantity, ranging from zero at the North pole to 180° at the South pole.

# Spherical harmonics visualization

In [2]:
from visualization.spherical_harmonics_visualization import visualizeAllHarmonicsOfDegree

visualizeAllHarmonicsOfDegree(1,resolution=20)

1 -1
Drawing ...


1 0
Drawing ...


C:\Users\ressl\Desktop\MSc Project\AlesMScProject\visualization\spherical_harmonics_visualization.py:21: RuntimeWarning: invalid value encountered in divide
  fcolors = (fcolors - fmin)/(fmax - fmin)


1 1
Drawing ...


# Spherical harmonics expansion

Any well-behaved function $f(\theta,\phi)$ on the sphere can be expanded to spherical harmonics basis as follows (Eq 5):

$$f(\theta, \phi) = \sum_{l=0}^{\infty }\sum_{m=-l}^{l}c^{m}_{l}Y^{m}_{l}(\theta, \phi)$$

where $c^{m}_{l}$ is the expansion coefficient for some spherical harmonic $Y^{m}_{l}(\theta, \phi)$ defined as (Eq 6):

$$c^{m}_{l} = \int_0^{2\pi} d\phi  \int_0^{\pi} d\theta sin(\theta) Y_{l}^{m*} (\theta,\phi) f(\theta,\phi)$$

Smooth functions with negligible high angular frequency content can be approximated with little to no loss by using harmonics only up to degree $l_{max}$ (Eq 7):

$$f(\theta, \phi) = \sum_{l=0}^{l_{max}}\sum_{m=-l}^{l}c^{m}_{l}Y^{m}_{l}(\theta, \phi)$$

Since the dMRI signals are real-valued and antipodlly symmetric (i.e. symmetric about the origin $f(x)=f(-x)$) simplified spherical harmonic basis can be used. The basis is real thus without imaginary components. Furthermore due to the atipodal symmetry all spherical harmonics with odd degree can be set to $0$. This is because the diffusion is symmetric about the origin e.g. diffusion in the direction $[1,1,1]$ is the same as diffusion in the direction $[-1,-1,-1]$. And since spherical harmonics have the following parity property (Eq 8):

$$Y^{m}_{l}(\theta,\phi)=(-1)^{m}\overline{Y^{-m}_{l}(\theta,\phi)}$$

and also (Eq 9)

$$Y^{m}_{l}(\pi-\theta,\pi+\phi)=(-1)^{l}Y^{m}_{l}(\theta,\phi)$$

it is now possible to see that only spherical hermonics with even degrees correspond to antipodal terms (the other spherical harmonics that are not symmetric around the origin are not needed in our case). Thus the resulting simplified basis becomes (Eq 10):

$$\begin{split}Y^{m}_{l}(\theta,\phi) = \begin{cases}
0 & \text{if $l$ is odd}, \\
\sqrt{2} \: \text{Im} \left[ Y_l^{-m}(\theta,\phi) \right] & \text{if $m < 0$},\\
Y_l^0(\theta,\phi) & \text{if $m = 0$},\\
\sqrt{2} \: \text{Re} \left[ Y_l^m(\theta,\phi) \right] & \text{if $m > 0$},\\
\end{cases}\end{split}$$


------------NOTE------------

The real spherical harmonics $Y^{m}_{l}$ with $m > 0$ are said to be of cosine type, and those with $m < 0$ of sine type. The main difference between sine-type and cosine-type spherical harmonics is the way they depend on the azimuthal angle. Cosine-type spherical harmonics are even functions with respect to $\phi$, while sine-type spherical harmonics are odd functions with respect to $\phi$.

----------------------------

# Least-squares expansion coefficient approximation

Even though the theoretical results above are valid, the coefficients of the spherical harmonics expansion need to be computed only from samples of $f(\theta,\phi)$ since the function is unknown. One possibility is to use Least-Squares approximation to compute the spherical harmonics expansion. However this method makes assumptions about the sampling grid (namely lat-lon sampling - Driscoll Healy). See the sampling schemes below:

![SPHERE_SAMPLING_SCHEMES](./figures/sphere_sampling_schemes.png)

However what is really needed is spherical harmonic expansion that closely approximates the sample points with the assumption that the function that passes near the sample points will also closely approximate the original function.

Let spherical functions ($L^{2}(S^{2})$) be the Hilbert space of square integrable functions on the 2-dimensional sphere $S^{2}$ and the coordinates are defined as colatitude $\theta$ and longitude $\phi$. Square integrable functions need to satisfy this condition (Eq 11):

$$\int_{-\infty }^{\infty }\left | f(x) \right |^{2}dx < \infty$$

where $f(x)$ is a real- or complex-valued function.

The inner product on the sphere is defined as (Eq 12):

$$
\left \langle f,h \right \rangle=\int_{0}^{\pi}\int_{0}^{2\pi}f(\theta,\phi)\overline{h(\theta,\phi)}\, sin\theta \, d\phi d\theta
$$

Thus the expansion coefficients can also be defined as (Eq 13):

$$\widehat{f}(l,m)=\left \langle f,Y^{m}_{l} \right \rangle=c^{m}_{l}$$

Since it is not possible to discretize the surface of the sphere in a way that the neigbourhood of each point is the same therefore a common approach to performing a spherical convolution is to express the discretized spherical function and a filter in the spherical harmonics domain and perform the convolution there. This approach is supported by the Sampling Theorem, Convolution Theorem and fast spherical transform. The Sampling Theorem guarantees the reversibility of the discretization of the spherical functions. And the fast spherical transform ensures computational efficiency.

## The Convolution Theorem
The Convolution Theorem states that expansion coefficient of a convolution of some spherical functions $f$ and $h$ is a product of the expansion coefficients of $f$ and $h$ (Eq 14):

$$(\widehat{f*h})(l,m)=2\pi\sqrt{\frac{4\pi}{2l+1}}\widehat{f}(l,m)\widehat{h}(l,0)$$

Since this theorem is independent of sampling it is therefore possible to perform convolution in the spherical harmonics domain as long as the projection of the functions onto the sperical harmonics domain is accurate. Due to this it is possible to transform the spherical functions $f$ and $h$ into the spherical harmonics domain, perform convolution there and the perform the inverse transform to get the convolution $f*h$.

## The Sampling Theorem
The Sampling Theorem states that for a bandlimited ($\widehat{f}(l,m)=0$ for $l\geq b$) spherical function $f$ of bandwidth $b$ the expansion coefficients can be calculated as (Eq 15):

$$\widehat{f}(l,m)=\frac{\sqrt{2\pi}}{2b}\sum_{j=0}^{2b-1}\sum_{k=0}^{2b-1}a^{(j)}_{b}f(\theta_{j},\phi_{k})\overline{Y^{l}_{m}(\theta_{j},\phi_{k})}$$

where $l<b$, $\left | m \right |\leq l$, $\theta_{j}=\pi\frac{2j+1}{4B}$, $\theta_{k}=\frac{\pi k}{b}$ and $a^{(b)}_{j}$ are weights that compensate for the oversampling at the poles. It is important to note that according to the theorem $(2b)^{2}$ samples are sufficient to be able to reconstruct $f$. However the assumption of this theorem is that the sphere is sampled as a lat-lon grid.

## The Uniform Resolution Theorem
The Uniform Resolution Theorem states that each spherical harmonic of some degree $l$ can be rotated and then expressed as a linear combination of only the harmonics of the same degree $l$. Therefore the impossibility of sphere discretization can be circumvented.

------------NOTE------------

Hilbert space: https://www.youtube.com/watch?v=_kJUUxjJ_FY

----------------------------

## Dataset

At first the dataset described in the "02 dMRI data" notebook is loaded.

In [3]:
bvals, qhat, dwis = loadhcp.load_hcp()

dwis.shape

(108, 145, 174, 145)

Plot the loaded dataset for one voxel.

In [4]:
%matplotlib qt

voxel = dwis[:,100,75,71]

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(qhat[0],qhat[1],qhat[2],c=voxel,vmin=1000,vmax=2000)
plt.show()

## Pre-processing

Measurements with $b=0$ need to be removed because they do not lie on the unit sphere.

In [5]:
bvals, qhat, dwis = remove_b_0_measurements(bvals, qhat, dwis)

Since the dMRI data is antipodal it is possible to reflect the data points through the origin to produce more samples.

In [6]:
bvals, qhat, dwis = extend_dataset_with_origin_reflections(bvals, qhat, dwis)

The following cell plots the augmented dataset for one voxel.

In [7]:
voxel = dwis[:,100,75,71]

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.scatter(qhat[0],qhat[1],qhat[2],c=voxel,vmin=1000,vmax=2000)
plt.show()

Transform Cartesian coordinates to spherical

In [8]:
thetas, phis = convert_coords_from_cartesian_to_spherical(qhat)

## Spherical Fourier transform

Even thought the data is not sampled on a lat-lon grid but rather uniformly it can still be assumed that the spherical harmonic series will approximate it closely. This is because with enough samples the conditions of sampling theorem are likely to be completely or nearly satisfied and thus the reconstruction of the function from the spherical harmonic series should be close to the original function.

Since each sample should be expressed as a linear combination of spherical harmonics evaluated at the corresponding coordinates, it is possible to construct an $n \times [0.5(l + 1)(l + 2)]$ design matrix $B$ where each row contains evaluations of the spherical harmonics up to the maximum degree $l$ evaluated at a given samples coordinates. The ordering of the spherical harmonics in the matrix row is determined by the formula $i=0.5l(l+1)+m$ where $i$ is the index.

The expansion coefficients are then determined by the least-squares fit (pseudo-inverse) as follows (Eq 18):

$$\overrightarrow{c}=(B^{T}B)^{-1}B^{T}\overrightarrow{s}$$

where $\overrightarrow{c}$ is the column vector of expansion coefficients and $\overrightarrow{s}$ is the column vector of samples. The inconvenience with least-squares is that there is no guarantee that $B^{T}B$ is going to be invertible. However when the sampling is uniform it is predominantly invertible but still without guarantee.

The matrix $(B^{T}B)^{-1}B^{T}$ is also referred to as a spherical fourier transform. The following cells computes it.

In [9]:
design_matrix = get_design_matrix(max_degree = 14,number_of_samples=len(bvals), thetas=thetas, phis=phis)
spherical_fourier_transform = get_spherical_fourier_transform(design_matrix)

Now the spherical fourier transform is used to determine the expansion coefficients.

In [10]:
expansion_coefficients = spherical_fourier_transform @ voxel
expansion_coefficients

array([ 5.82287434e+03,  1.95954714e+02, -3.82470462e+02,  1.06431203e+02,
        2.39077501e+02,  8.13529067e+02, -9.38223661e+01,  5.27989304e+01,
        3.42160645e+01, -1.48454684e+02, -5.46924338e+01, -5.92998601e+01,
       -4.03104470e+01,  3.20675100e+01,  1.32335074e+02, -4.70212064e+01,
        3.52080731e+01, -1.16102696e+02,  4.35006832e+01, -3.98042189e+01,
        3.21692442e+01, -3.32552973e+01,  9.86793909e+01, -2.54006665e+01,
       -9.45103211e+01, -1.16018225e+00,  3.05453764e+01,  1.44061337e+02,
        8.29368993e+01, -8.60597789e+01,  8.78662768e+01,  5.57905991e+00,
        4.96641972e+01,  5.28666013e+01, -1.36560937e+01,  3.84349509e+00,
       -1.06849134e+02, -2.42287493e+02,  1.06473601e+02, -9.64867508e+01,
        1.17661362e+01, -1.27204123e+02,  3.58878684e+01,  3.35414134e+01,
        1.47878681e+02,  1.63100736e+02, -1.87236112e+01,  2.33320064e+01,
        5.02750333e+01, -6.64969432e+01,  2.54784461e+01,  7.63297198e+01,
       -2.28423444e+02, -

## Signal reconstruction

The original signal can be reconstructed by inverse spherical fourier transform which is equivalent to $B$.

In [21]:
inverse_spherical_fourier_transform = get_inverse_spherical_fourier_transform(design_matrix)

original_signal = dwis[:,100,75,71]
reconstructed_signal = inverse_spherical_fourier_transform @ expansion_coefficients

In [22]:
original_signal

array([1120.1832 , 1746.1273 , 1994.284  , 2355.8635 , 1141.3386 ,
       1319.1646 , 1692.9148 , 1304.5999 , 1776.4893 , 1850.2423 ,
       1736.5635 , 1959.4779 , 1536.9711 , 1241.8085 , 2115.1946 ,
       2048.0872 , 1764.5569 , 2015.017  , 1919.6414 , 1652.7312 ,
       1700.7474 , 1147.5239 , 1885.3145 , 1635.8328 , 1400.7739 ,
       1456.9565 , 1014.1146 , 2092.2744 , 1850.6425 , 1398.3287 ,
       1888.2919 , 1602.9725 , 1717.0581 , 1500.4734 , 1317.569  ,
       1835.3228 , 2104.8643 , 1655.3278 , 1379.6798 , 1750.504  ,
       1587.1937 , 1978.541  ,  959.55365, 2120.06   , 1500.2799 ,
       1867.4877 , 1789.7793 , 1671.7902 , 1530.8884 , 1508.7181 ,
       1415.8063 , 1082.3871 , 2552.8623 , 1864.1335 , 2047.2896 ,
       1640.3965 , 1325.0327 , 1245.0833 , 1847.978  , 1652.5964 ,
       1982.6775 , 1707.3073 , 1818.8319 , 1844.2772 , 1291.6127 ,
       1794.27   , 1594.973  , 2286.7366 , 1277.6061 , 1484.5334 ,
       2290.4087 , 1051.1519 , 1905.5702 , 1946.6218 , 1586.50

In [23]:
reconstructed_signal

array([1120.18322754, 1746.12731934, 1994.28405762, 2355.86352539,
       1141.33862305, 1319.16455078, 1692.91479492, 1304.59985352,
       1776.48925781, 1850.24230957, 1736.56347656, 1959.47790527,
       1536.97106934, 1241.80847168, 2115.19458008, 2048.0871582 ,
       1764.55688477, 2015.01696777, 1919.64135742, 1652.73120117,
       1700.74743652, 1147.52392578, 1885.31445312, 1635.83276367,
       1400.77392578, 1456.95654297, 1014.11462402, 2092.27441406,
       1850.64245605, 1398.32873535, 1888.29187012, 1602.97253418,
       1717.05810547, 1500.47338867, 1317.56896973, 1835.32275391,
       2104.86425781, 1655.32775879, 1379.67980957, 1750.50402832,
       1587.19372559, 1978.54101562,  959.5536499 , 2120.06005859,
       1500.27990723, 1867.4876709 , 1789.77929687, 1671.79016113,
       1530.88842773, 1508.71813965, 1415.80627441, 1082.38708496,
       2552.86230469, 1864.13354492, 2047.28955078, 1640.39648437,
       1325.03271484, 1245.08325195, 1847.97802734, 1652.59643

The following cell shows the reconstruction error for each sample.

In [24]:
original_signal-reconstructed_signal

array([-1.36424205e-12,  3.63797881e-12, -1.81898940e-12, -2.72848411e-12,
       -5.22959454e-12,  3.86535248e-12,  2.95585778e-12, -1.36424205e-12,
        1.36424205e-12,  5.00222086e-12,  4.77484718e-12, -1.81898940e-12,
        9.09494702e-13, -4.32009983e-12, -1.81898940e-12,  3.63797881e-12,
       -9.09494702e-13, -4.09272616e-12, -4.54747351e-12, -2.04636308e-12,
       -6.36646291e-12,  1.81898940e-12,  1.81898940e-12, -6.82121026e-13,
       -3.18323146e-12,  2.04636308e-12,  1.47792889e-12, -3.18323146e-12,
       -3.86535248e-12,  1.36424205e-12, -6.82121026e-12, -5.22959454e-12,
        6.36646291e-12, -3.18323146e-12,  2.27373675e-12, -9.09494702e-13,
        3.18323146e-12, -1.13686838e-12, -1.36424205e-12, -5.00222086e-12,
        1.13686838e-12,  2.27373675e-13,  4.54747351e-13,  9.09494702e-13,
       -3.18323146e-12,  4.54747351e-13,  4.54747351e-12, -5.91171556e-12,
       -8.18545232e-12, -1.81898940e-12, -1.59161573e-12,  0.00000000e+00,
       -2.72848411e-12, -